<a href="https://colab.research.google.com/github/MMoronto/ml-unstructured-data-projects/blob/master/Tile_based_classification_using_Sentinel_2_L1C_and_EuroSAT_data_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Introduction**

This workflow explores the process of infering land use / classification from a pre-trained model based on the benchmark EuroSAT dataset. This note book contains notes I've taken as I work through the example workflows presented in the AI for Earth monitoring MOOC on Futurelearn.

This is the second of a two part practice module, the first part of this two part series explores the process of training a convolutional neural network based on EuroSAT benchmark data for land use/land cover classification while the second part explores the workflow for utilising the learned capabilities of our pre-trained model to infer land use / land cover classification on a Sentinel-2 level-1C tile image.   

## **Data**

The inference process utilizes the following data:

* a `Sentinel-2 level-1C file`, which is available in the following folder: ./S2_Tile_based_classification/01_input_data/S2_tile_4_inference.
The scene shows a coasteal part over Italy on March 31, 20121. The scene is used as input data for the pretrained model in order to infer land use/land cover classes.
* a `pretrained  model`, a `convolutional neural network` which has been trained based on EuroSAT  data, as a benchmark dataset for land use / land cover classifications.

## **Additional Resources**

* 3B - Tile-based classification with EuroSAT data - Training
* EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification
* EuroSAT data

## **Notebook Outline**

* 1 - Load a Sentinel-2 Level 1-C tile
* 2 - Resample all bands of a Seninel-2 Level 1-C tile to 10m spatial resolution
* 3 - Reorder the bands according to the order of the pretrained model
* 4 - Load the pretrained sequential convolutional neural network based on EuroSAT data
* 5 - Divide the Sentinel-2 L1C tile into  64x64 windows
* 6 - Infer land use classes
* 7 - Visualize the final classified image

# **1. Load a Sentinel-2 Level-1C tile**

**Import libraries**

In [ ]:
## Begin S3FS import snippet ##
import os, sys
s3_home = os.getcwd()
try: sys.path.remove(s3_home) # Remove the S3 root from the path
except Exception: pass

# Begin imports #
import tensorflow as tf
from osgeo import gdal_array, osr, gdal
import glob
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.colors
import warnings
warnings.,filterwarnings("ignoe")
# end imports #

os.chdir(current_dir) # go back to your previous directory

sys.path.append(s3_home) # restore the s3 root in the path

## end s3fs import snippet ##

#**2. Resample all bands of a Seninel-2 Level 1-C tile to 10m spatial resolution**

#**3. Reorder the bands according to the order of the pretrained model**

#**4. Load the pretrained sequential convolutional neural network based on EuroSAT data**


#**5. Divide the Sentinel-2 L1C tile into  64x64 windows**

#**6. Infer land use classes**

#**7. Visualize the final classified image**